In [158]:
from sympy import *
import numpy as np
from scipy.stats import norm

In [159]:
N = norm(0, 1)

### d1 & d2 for blackscholes

In [162]:
"""Args:
    s (float): underlying spot price
    k (float): strike price
    r (float): continuous risk free rate
    q (float): continuous dividend yield
    v (float): return volatility
    t (float): time to expiry 
"""

#d1

def d1(s, k, r, q, v, t):

    if t == 0:
        raise ValueError("Time to expiry (t) must be greater than 0.")

    a = (np.log(s / k) + (r - q + 0.5 * v**2) * t) / (v * np.sqrt(t))

    return a

#d2

def d2(s, k, r, q, v, t):

    b = d1(s, k, r, q, v, t) - v * np.sqrt(t)

    return b

In [163]:
"""Moneyness indicator

    Args:
        s (float): underlying price
        k (float): strike price
        is_call (bool): True if is call option else False if put

"""

def itm(s, k, is_call):

    if is_call:
        i = s>k
    else:
        i = s<k

    return i

In [164]:
"""Payoff function

    Args:
        s (float): underlying price
        k (float): strike price
        is_call (bool): True if is call option else False if put

    Returns:
        float: the payoff
"""

def payoff(s: float, k: float, is_call: bool) -> float:

    if is_call:
        poff = max(s - k, 0)
    else:
        poff = max(k - s, 0)

    return poff 

### Price

In [165]:
"""Black-Scholes price

    Args:
        s (float): underlying spot price
        k (float): strike price
        r (float): continuous risk free rate
        q (float): continuous dividend yield
        v (float): return volatility
        t (float): time to expiry
        is_call (bool): True if is call option else False if put

    Returns:
        float: Black-Scholes price
"""

def price(s, k, r, q, v, t, is_call):

    D1 = d1(s, k, r, q, v, t)
    D2 = d2(s, k, r, q, v, t)

    if is_call:
        p = s * np.exp(-q * t) * N.cdf(D1) - k * np.exp(-r * t) * N.cdf(D2)
    else:
        p = k * np.exp(-r * t) * N.cdf(-D2) - s * np.exp(-q * t) * N.cdf(-D1)

    return p

### Delta

In [166]:
"""Black-Scholes delta

    Args:
        s (float): underlying spot price
        k (float): strike price
        r (float): continuous risk free rate
        q (float): continuous dividend yield
        v (float): return volatility
        t (float): time to expiry
        is_call (bool): True if is call option else False if put

    Returns:
        float: Black-Scholes delta
"""

def delta(s, k, r, q, v, t, is_call):
    
    D1 = d1(s, k, r, q, v, t)

    if is_call:
        d =  np.exp(-q * t) * N.cdf(D1)
    else:
        d =  -np.exp(-q * t) * N.cdf(-D1)
    
    return d

### Delta

In [167]:
"""Black-Scholes theta

    Args:
        s (float): underlying spot price
        k (float): strike price
        r (float): continuous risk free rate
        q (float): continuous dividend yield
        v (float): return volatility
        t (float): time to expiry
        is_call (bool): True if is call option else False if put

    Returns:
        float: Black-Scholes theta
"""

def theta(s, k, r, q, v, t, is_call):

    D1 = d1(s, k, r, q, v, t)
    D2 = d2(s, k, r, q, v, t)

    if t == 0:
        raise ValueError("Time to expiry (t) must be greater than 0.")

    term1 = -(s * v * np.exp(-q * t) * N.pdf(D1)) / (2 * np.sqrt(t))

    # Implementing separate formulas for Call and Put
    if is_call:
        t = (term1 + q * s * np.exp(-q * t) * N.cdf(D1) - r * k * np.exp(-r * t) * N.cdf(D2))
    else:
        t =  (term1 - q * s * np.exp(-q * t) * N.cdf(-D1) + r * k * np.exp(-r * t) * N.cdf(-D2))

    return t

### Gamma

In [168]:
def gamma(s, k, r, q, v, t):

    if t == 0:
        raise ValueError("Time to expiry (t) must be greater than 0.")

    D1 = d1(s, k, r, q, v, t)

    g = (np.exp(-q * t) * N.pdf(D1)) / (s * v * np.sqrt(t))

    return g

### Vega

In [169]:
def vega(s, k, r, q, v, t):

    D1 = d1(s, k, r, q, v, t)

    v = s * np.exp(-q * t) * N.pdf(D1) * np.sqrt(t)

    return v

### Rho

In [170]:
def rho(s, k, r, q, v, t, is_call):

    D2 = d2(s, k, r, q, v, t)

    if is_call:
        r_val = k * t * np.exp(-r * t) * N.cdf(D2)
    else:
        r_val = -k * t * np.exp(-r * t) * N.cdf(-D2)

    return r_val


In [171]:
def call_price(s, k, r, q, v, t):

    c = price(s, k, r, q, v, t, is_call=True)

    return c 

def put_price(s, k, r, q, v, t):

    p = price(s, k, r, q, v, t, is_call=False)

    return p

In [172]:
def call_delta(s, k, r, q, v, t):

    cd = delta(s, k, r, q, v, t, is_call=True)

    return cd

def put_delta(s, k, r, q, v, t):

    pd = delta(s, k, r, q, v, t, is_call=False)

    return pd

In [173]:
def call_theta(s, k, r, q, v, t):

    ct = theta(s, k, r, q, v, t, is_call=True)

    return ct

def put_theta(s, k, r, q, v, t):

    pt = theta(s, k, r, q, v, t, is_call=False)

    return pt

In [174]:

def call_rho(s, k, r, q, v, t):

    cr = rho(s, k, r, q, v, t, is_call=True)
    
    return cr


def put_rho(s, k, r, q, v, t):

    pr = rho(s, k, r, q, v, t, is_call=False)
    
    return pr

### Test 1

In [242]:
s = 75
k = 50
r = 0.80
q = 0.02
v = 0.35
t = 2.5

In [244]:
print('Call Price = ', call_price(s, k, r, q, v, t))
print('Call Rho = ', call_rho(s, k, r, q, v, t))
print('Call Delta = ', call_delta(s, k, r, q, v, t))
print('Call Theta = ', call_theta(s, k, r, q, v, t))
print('Gamma = ', gamma(s, k, r, q, v, t))
print('Vega = ', vega(s, k, r, q, v, t))

Call Price =  64.57546879972466
Call Rho =  16.916326661912844
Call Delta =  0.9512266595265307
Call Theta =  -3.98649324559157
Gamma =  3.155100422843393e-07
Vega =  0.0015529009893682326


### Test 2

In [238]:
s = 40
k = 65
r = 0.08
q = 0.03
v = 0.25
t = 2.5

In [240]:
print('Put Price = ', put_price(s, k, r, q, v, t))
print('Put Rho = ', put_rho(s, k, r, q, v, t))
print('Put Delta = ', put_delta(s, k, r, q, v, t))
print('Put Theta = ', put_theta(s, k, r, q, v, t))
print('Gamma = ', gamma(s, k, r, q, v, t))
print('Vega = ', vega(s, k, r, q, v, t))

Put Price =  17.81282311787468
Put Rho =  -115.27280992710939
Put Delta =  -0.7074075213242269
Put Theta =  1.9330234824712536
Gamma =  0.018136348192143496
Vega =  18.136348192143497
